#### Importing relevant packages

In [1]:
# To display full output in Notebook, instead of only the last result 
from IPython.core.interactiveshell import InteractiveShell 

InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from bs4 import BeautifulSoup #requires pip install
import requests #requires pip install

In [3]:
#Test zoopla scraper
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup #requires pip install
import requests
import re
from re import sub
from decimal import Decimal
import io
from statistics import mode

# Convert price string into a numerical value
def to_num(price):
    value = Decimal(sub(r'[^\d.]', '', price))
    return float(value)

def is_dropped(money):
    for i in range(len(money)):
        if(money[i] != '£' and money[i] != ',' and (not money[i].isdigit())):
            return True
    return False


In [26]:
#Dexter scraper:

# Import packages
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup #requires pip install
import requests #requires pip install
import re
import time

import io
# document time
time_started = str(datetime.datetime.now()).replace(" ","_").replace(":","-")[0:19]
## Define list of subway stations
Underground_lines = ['Bakerloo', 'Central', 'Circle', 'District', 'DLR', 'Hammersmith & City',
                 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']

## Function to extract characteristics on each ad from the main webpage
def feature_extract(html_text):

    soup = BeautifulSoup(html_text, 'lxml')

    ## Parse for the different divisions within the add

    # ads = soup.find_all('div', class_ = 'result-content') #searches for 'div' and is filtered by the CSS-snippet
    ads = soup.find_all('li', class_ = 'result item for-sale infinite-item')#searches for 'div' and is filtered by the CSS-snippet
    ## Set-up for the loop
    results = {} #create nested dictionary to store the results
    id_ad = 0 #insert ad_ID to distinguish between each ad

    ## Loop across all ads
    for k in range(len(ads)):
        ad = ads[k]
        id_ad += 1
        results[id_ad] = {}

        ## Extracting features from the ad
        name = ad.find('h3').a.contents[0]
        try:
            price = ad.find('span', class_ = 'price-qualifier').text #catches the price WITHIN one ad
        except:
            continue
        address = ad.find('span', class_ = 'address-area-post').text

        # Number of bedrooms extracted from string
        try:
            bedrooms = ad.find('li', class_ = 'Bedrooms').text
        except:
            continue
        bedrooms_nbr = int(bedrooms.split()[0])

        # Number of bedrooms extracted from string
        bathrooms_str = str(ad.find('li',class_ = 'Bathrooms'))
        bathrooms_nbr = re.findall(r'\d+', bathrooms_str)
        bathrooms_nbr2 = int(bathrooms_nbr[0] if len(bathrooms_nbr)!= 0  else 0)

        # Number of bedrooms extracted from string
        reception_str = str(ad.find('li',class_ = 'Receptions'))
        reception_nbr = re.findall(r'\d+', reception_str)
        reception_nbr2 = int(reception_nbr[0] if len(reception_nbr)!= 0  else 1)

        link = ad.find('h3').a.get("href")

        ad_identification = ads[k]['data-property-id']

        # Create dictionary of results per ad id
        results[id_ad]['ad_identification'] = ad_identification
        results[id_ad]["street_name"] = name
        results[id_ad]["price"] = price
        results[id_ad]["address"] = address
        results[id_ad]["bedrooms"] = bedrooms_nbr
        results[id_ad]["bathrooms"] = bathrooms_nbr2
        results[id_ad]["reception"] = reception_nbr2
        results[id_ad]["link"] = ("https://www.dexters.co.uk" + link)

        # Create dataframe from dictionary of results
        df_houses = pd.DataFrame.from_dict(results, orient='index')

    return df_houses

## Function to create list of pages base on url and number of iterations desired
def page_list(string, iterations):
    pages_list = []
    for i in range(iterations):
        pages_list.append(string + str(i+1))

    return pages_list

## Function to get the maximum number of listing on Dexter's website
def page_max(url):
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    amount = soup.find('span', class_ = 'marker-count has-results').text
    amount_num = re.sub('\D', '', amount)
    return int(amount_num)

## Function to launch scrapper on a specific webpage with number of pages to scrap
def pages_scrap(main_page, iter_page, pages):
    max_pages = (page_max(main_page)/18)
    list_of_pages = page_list(iter_page, pages) # Create list of pages to scrape
    df_list = [] #Create list of dataframes to be concatenated by the end of the loop

    # Loop through all pages to create the different dataframes
    for page in list_of_pages:
        html_page = requests.get(page)
        html_page.encoding = 'utf-8'
        page = html_page.text
        df_ads = feature_extract(page)
        df_list.append(df_ads)

    # Concatenate the different dataframes
    df_results = pd.concat(df_list)
    df_results = df_results.drop_duplicates()
    df_results = df_results.reset_index(drop=True)

    print('Remaining number of page: ', int(max_pages - pages) )

    return df_results
# 1.2 Subway related functions

## Function to extract subway info list from a house webpage on dexter
def get_info_subway(link):
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')
    subway = soup.find('ul', class_ = 'list-information').text

    return subway

## Function to get list of values for subway distances with string
def sub_values(string):
    split = string.split('\n')
    list_1 = list(filter(None, split))

    list_2 = []
    for i in list_1:
        x = i.split('-')
        list_2.append(x)

    list_3 = [item.strip() for sublist in list_2 for item in sublist]
    list_4 = list_3[0:3]

    return list_3

## Function to get the closest stop on the tube if any
def closest_line(list_of_lines):
    j = 0
    nearby_data = []
    for i in range(len(list_of_lines)):
        if list_of_lines[i] == 'London Underground' or list_of_lines[i] in Underground_lines and (j != 1 and i!=0):
            if (' ' in list_of_lines[i-2]) == False :
                nearby_data.append(list_of_lines[i-3])
                nearby_data.append(list_of_lines[i-2])
                nearby_data.append(list_of_lines[i-1])
                nearby_data.append(list_of_lines[i])
                j = 1

                nearby_data[0] = (' '.join(nearby_data[0:2]))
                del nearby_data[1]

            else:
                nearby_data.append(list_of_lines[i-2])
                nearby_data.append(list_of_lines[i-1])
                nearby_data.append(list_of_lines[i])
                j = 1

    return nearby_data

## Function to populate datafrmae with closest tube stop name, distance, and related tube line
def subway_per_house(df):
    #Create new empty (NaN) columns in the existing dataframe
    df = df.reindex(columns = df.columns.tolist() + ['subway_station','distance','tube_line'])

    #Loop through all lines of dataframe
    for i in range(len(df)):
        x = df['link'].iloc[i] #Get link of house page to scrape
        subs = get_info_subway(x) #Extract tube line info
        subs_2 = sub_values(subs) #Get list of subway station and distance
        subs_3 = closest_line(subs_2) #Extract closest tube station only

        # Populate dataframe if a tubeway station has been found or not
        if len(subs_3)!= 0:
            df['subway_station'].iloc[i] = subs_3[0]
            df['distance'].iloc[i] = subs_3[1]
            df['tube_line'].iloc[i] = subs_3[2]
        else:
            df['subway_station'].iloc[i] = np.NaN
            df['distance'].iloc[i] = np.NaN
            df['tube_line'].iloc[i] = np.NaN

    df = df.astype(str)

    return df

#Functions to clean subway output
def get_tube_dist(string):
    string_m = string.split(' ')
    num_val = string_m[-1]

    return num_val
def strip_tube(string):
    string_m = string.split(' ')
    string_m = string_m[:-1]
    string_m = ' '.join(string_m)

    return string_m
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

## Function to clean subway stops when too many words in the string
def clean_tube_stop_string(string):
    forbiddden_words = ['London Overground', 'Railway', 'Network Rail', 'Tramlink']
    count_forbidden = 0

    for j in forbiddden_words:
        if count_forbidden == 0:
            if j in string:
                string_update = string.split()[-1]
                count_forbidden = 1
            else:
                string_update = string

    return(string_update)

## Function to input tube distance into the right column when value is in 'tube_stop'
def clean_tube_dist(df):
    df['distance'] = df['distance'].astype('str')

    errors  = df[df.loc[:, 'distance'].map(hasNumbers) == False].copy()
    errors_2 = errors.loc[errors['subway_station'] != 'NaN'].copy()
    errors_2.loc[:, 'distance'] = errors_2.loc[:, 'subway_station'].map(get_tube_dist)
    errors_2.loc[:, 'subway_station'] = errors_2.loc[:, 'subway_station'].map(strip_tube)
    errors_2

    #Create copy of original df for modification
    df_copy = df.copy()

    # replace values in final df
    for i in errors_2.index:
        df_copy.loc[i] = errors_2.loc[i]

    return df_copy

## Functions to deal with Victoria tube stops (Victoria being both a tube stop and a tube line)
def victoria_clean_stop(string):
    str_vic = 'Victoria'
    str_check = string.split()
    if str_check[0] == 'Victoria':
        str_return = str_check[1]
    else:
        str_return = str_vic

    return str_return
def clean_tube_victoria(df):
    df['subway_station'] = df['subway_station'].astype('str')

    errors  = df[df['subway_station'].str.contains('Victoria')].copy()

    errors.loc[:, 'subway_station'] = errors.loc[:, 'subway_station'].map(victoria_clean_stop)

    #Create copy of original df for modification
    df_copy = df.copy()

    # Replace values in final df
    for i in errors.index:
        df_copy.loc[i] = errors.loc[i]

    return df_copy

## Final cleaning function to apply previous cleaning on 'tube_stop' and 'tube_dist' for the whole dataframe
def clean_tube_stop(df):
    df_2 = df.copy()
    df_2 = clean_tube_dist(df_2)
    df_2['subway_station'] = df_2['subway_station'].astype('str')
    df_2['subway_station'] = df_2['subway_station'].map(clean_tube_stop_string)

    df_2 = clean_tube_victoria(df_2)
    # #Keep the ID of the add as index or not


    return df_2

dexter_list_1 = pages_scrap('https://www.dexters.co.uk/property-sales/properties-for-sale-in-london',
                            'https://www.dexters.co.uk/property-sales/properties-for-sale-in-london/page-', 50)


## Fetch subway related information from the previous dataframe
output_list = subway_per_house(dexter_list_1)


cleaned = clean_tube_stop(output_list)
cleaned

Remaining number of page:  213


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ad_identification,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,tube_line
0,151907,Lancaster Gate,"£23,000,000","Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26m,Central
1,124656,Wilton Crescent,"£19,500,000","Belgravia, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22m,Piccadilly
2,138546,Whitehall Place,"£9,250,000","Whitehall, SW1A",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
3,143179,Old Queen Street,"£9,250,000","Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16m,Circle
4,138545,Whitehall Place,"£9,000,000","Whitehall, SW1A",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
...,...,...,...,...,...,...,...,...,...,...,...
339,146752,Duchess Walk,"£1,300,000","Shad Thames, SE1",2,2,1,https://www.dexters.co.uk/property-for-sale/fl...,Tower Hill,0.46m,District
340,147283,Kingsland Road,"£1,300,000","Bethnal Green, E2",2,2,1,https://www.dexters.co.uk/property-for-sale/fl...,Old Street,0.53m,Northern
341,152598,Osborn Street,"£1,300,000","Shoreditch, E1",3,2,1,https://www.dexters.co.uk/property-for-sale/pr...,Aldgate East,0.14m,District
342,155207,Hackney Road,"£1,300,000","Hackney, E2",3,2,2,https://www.dexters.co.uk/property-for-sale/ho...,Bethnal Green,0.47m,Central


In [27]:
cleaned.shape

(344, 11)

In [28]:
%%time
#get the results of EPC ratings for all the street names in the Dexter scraper:

#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in list(cleaned['street_name']):
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)

    #iterate through
for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(None)
avg_epc_tuple=list(zip(list(cleaned['street_name']),epc_avg))
street_name_epc=pd.DataFrame(avg_epc_tuple, columns=['street_name','average_epc'])
street_name_epc

,street_name,average_epc
0,Lancaster Gate,C
1,Wilton Crescent,D
2,Whitehall Place,D
3,Old Queen Street,C
4,Whitehall Place,D
...,...,...
339,Duchess Walk,B
340,Kingsland Road,B
341,Osborn Street,D
342,Hackney Road,C


In [34]:
street_name_epc['average_epc'].isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
339    False
340    False
341    False
342    False
343    False
Name: average_epc, Length: 344, dtype: bool

In [172]:
%%time
#attempting to scrape the streets for the entire alphabet
list_london_street_names=[]
for letter in list(string.ascii_lowercase):

    london_streetmap='http://london.streetmapof.co.uk/'
    streetmap_url=london_streetmap+letter+'/'


    map_html_text = requests.get(streetmap_url).text
    soup_map = BeautifulSoup(map_html_text, 'lxml')

    #find street names in the map:
    map_ads=soup_map.find_all('td', { "valign": "top" })


    # ad = map_ads[1]
    count_pages=[]
    [count_pages.append(a) for a in soup_map.find_all('a', href=True) if a['href'].startswith(f'/{letter}/')]
    len(count_pages)

    list_starting_with_letter=[]
    if len(count_pages)>0:
        list_starting_with_letter=[]
        for i in range(len(count_pages)):
            if i==0:
                streetmap_url=london_streetmap+letter+'/'

                map_html_text = requests.get(streetmap_url).text
                soup_map = BeautifulSoup(map_html_text, 'lxml')
                map_ads=soup_map.find_all('td', { "valign": "top" })
                for k in range(len(map_ads)):
                    ad = map_ads[k]


                    for a in ad.find_all('a', href=True):
                        if a['href'].startswith(f'/{letter}/'):
                            count_pages.append(a)
                        else:
                            list_starting_with_letter.append(a.get_text(strip=True))
            else:
                streetmap_url=london_streetmap+letter+'/'+str(i+1)

                map_html_text = requests.get(streetmap_url).text
                soup_map = BeautifulSoup(map_html_text, 'lxml')
                map_ads=soup_map.find_all('td', { "valign": "top" })

                for k in range(len(map_ads)):
                    ad = map_ads[k]


                    for a in ad.find_all('a', href=True):
                        if a['href'].startswith(f'/{letter}/'):
                            count_pages.append(a)
                        else:
                            list_starting_with_letter.append(a.get_text(strip=True))
        len(list(set(list_starting_with_letter)))
        list_london_street_names.append(list(set(list_starting_with_letter)))
    else:
        list_starting_with_letter=[]
        streetmap_url=london_streetmap+letter+'/'

        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{letter}/'):
                    break
                else:
                    list_starting_with_letter.append(a.get_text(strip=True))
        len(list(set(list_starting_with_letter)))
        list_london_street_names.append(list(set(list_starting_with_letter)))
        
#flatten the list of lists for london street names
flatten = itertools.chain.from_iterable
list_london_street_names=list(flatten(list_london_street_names))
list_london_street_names

CPU times: user 7.64 s, sys: 725 ms, total: 8.37 s
Wall time: 1min 42s


['Avonmouth Street',
 'Ainger Mews',
 'Angell Road',
 'Aldred Road',
 'Alyth Gardens',
 'Argyll Road',
 'Arnold Circus',
 'Ashfield Yard',
 'Abbey Mews',
 'Arngask Road',
 'Atherton Road',
 'Abbey Lane',
 'Agincourt Road',
 'Annesley Road',
 'Appledore Close',
 'Ada Gardens',
 'Ambleside Road',
 'Ashley Walk',
 'Aylesbury Road',
 'Apsley Way',
 'Alkerden Road',
 'Auriol Road',
 'Arden Mews',
 'Aberdeen Park',
 'All Saints Road',
 'Amyruth Road',
 'Ash Tree Dell',
 'Ambleside Avenue',
 'Arthur Horsley Walk',
 'Ashbourne Way',
 'Acton Lane',
 'Arlesey Close',
 'Arlington Gardens',
 'Amber Avenue',
 'Arnhem Way',
 'Arthur Road',
 'Ashmount Estate',
 'Alfreton Close',
 'Ainsdale Road',
 'Aberford Gardens',
 'Albert Grove',
 'Aldbourne Road',
 'Allingham Close',
 'Angel Place',
 'Adam Close',
 'Aylett Road',
 'Axminster Road',
 'Ardshiel Close',
 'Acacia Road',
 'Archangel Street',
 'Allsop Place',
 'Augustine Road',
 'Awlfield Avenue',
 'Arbor Court',
 'Aldridge Road Villas',
 'Archbishops

In [184]:
df=pd.DataFrame(list_london_street_names, columns=['street_name'])

In [185]:
df

,street_name
0,Avonmouth Street
1,Ainger Mews
2,Angell Road
3,Aldred Road
4,Alyth Gardens
...,...
23332,Zoar Street
23333,Zenoria Street
23334,Zangwill Road
23335,Zetland Street


In [48]:
df.to_csv('london_street_names.csv', index=False, header=True)

In [181]:
#adding another column to the dataframe with all streets converted to links suitable for web scraping EPC ratings:
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in df['street_name'].tolist():
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)

df['URL']=np.array(list_epc_urls)
df

,street_name,URL
0,Avonmouth Street,https://find-energy-certificate.digital.commun...
1,Ainger Mews,https://find-energy-certificate.digital.commun...
2,Angell Road,https://find-energy-certificate.digital.commun...
3,Aldred Road,https://find-energy-certificate.digital.commun...
4,Alyth Gardens,https://find-energy-certificate.digital.commun...
...,...,...
23332,Zoar Street,https://find-energy-certificate.digital.commun...
23333,Zenoria Street,https://find-energy-certificate.digital.commun...
23334,Zangwill Road,https://find-energy-certificate.digital.commun...
23335,Zetland Street,https://find-energy-certificate.digital.commun...


In [149]:
df[0:5]

,street_name,URL
0,Avonmouth Street,https://find-energy-certificate.digital.commun...
1,Ainger Mews,https://find-energy-certificate.digital.commun...
2,Angell Road,https://find-energy-certificate.digital.commun...
3,Aldred Road,https://find-energy-certificate.digital.commun...
4,Alyth Gardens,https://find-energy-certificate.digital.commun...


In [150]:
%%time
#trying to iterate through the urls in the column:
df_temp=df['URL'][0:5]
for url in df_temp.tolist():
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
df_average_epc=pd.DataFrame(epc_avg, columns=['average_epc'])
df_average_epc
# avg_epc_tuple=list(zip(list_starting_withZ,epc_avg))
street_name_epc=pd.concat([df_temp, df_average_epc],axis=1)
street_name_epc

CPU times: user 195 ms, sys: 100 ms, total: 295 ms
Wall time: 40.1 s


,URL,average_epc
0,https://find-energy-certificate.digital.commun...,D
1,https://find-energy-certificate.digital.commun...,E
2,https://find-energy-certificate.digital.commun...,C
3,https://find-energy-certificate.digital.commun...,D
4,https://find-energy-certificate.digital.commun...,D


In [186]:
df.reset_index(level=0, inplace=True)
df=df.rename(columns={'index': 'street_id'})
df

,street_id,street_name
0,0,Avonmouth Street
1,1,Ainger Mews
2,2,Angell Road
3,3,Aldred Road
4,4,Alyth Gardens
...,...,...
23332,23332,Zoar Street
23333,23333,Zenoria Street
23334,23334,Zangwill Road
23335,23335,Zetland Street


In [5]:
#read the CSV which is a result of merging street_names from the Dexters scraper with the URLs to scrape the EPC ratings. Next steps is to actually scrape the average EPC
merged_table=pd.read_csv('merged_table.csv')
merged_table

,street_name,url
0,Fairhazel Gardens,https://find-energy-certificate.digital.commun...
1,Green Street,https://find-energy-certificate.digital.commun...
2,Kings Road,https://find-energy-certificate.digital.commun...
3,Lancaster Gate,https://find-energy-certificate.digital.commun...
4,Ladbroke Road,https://find-energy-certificate.digital.commun...
5,Old Queen Street,https://find-energy-certificate.digital.commun...
6,Philbeach Gardens,https://find-energy-certificate.digital.commun...
7,Queen Street,https://find-energy-certificate.digital.commun...
8,Queen Street,https://find-energy-certificate.digital.commun...
9,Wilton Crescent,https://find-energy-certificate.digital.commun...


In [7]:
%%time
#trying to iterate through the urls in the column:
epc_avg=[]
for url in merged_table['url'].tolist():
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
df_average_epc=pd.DataFrame(epc_avg, columns=['average_epc'])
street_name_epc=pd.concat([merged_table, df_average_epc],axis=1)
street_name_epc

CPU times: user 893 ms, sys: 47.8 ms, total: 941 ms
Wall time: 1min


,street_name,url,average_epc
0,Fairhazel Gardens,https://find-energy-certificate.digital.commun...,D
1,Green Street,https://find-energy-certificate.digital.commun...,D
2,Kings Road,https://find-energy-certificate.digital.commun...,D
3,Lancaster Gate,https://find-energy-certificate.digital.commun...,C
4,Ladbroke Road,https://find-energy-certificate.digital.commun...,D
5,Old Queen Street,https://find-energy-certificate.digital.commun...,C
6,Philbeach Gardens,https://find-energy-certificate.digital.commun...,C
7,Queen Street,https://find-energy-certificate.digital.commun...,C
8,Queen Street,https://find-energy-certificate.digital.commun...,C
9,Wilton Crescent,https://find-energy-certificate.digital.commun...,D


In [83]:
#FINAL for the bigger for loop to collect all the elements of the list for letter "Z" because it's a short list


london_streetmap='http://london.streetmapof.co.uk/'
alphabet_letter='a'
streetmap_url=london_streetmap+alphabet_letter+'/'

    
map_html_text = requests.get(streetmap_url).text
soup_map = BeautifulSoup(map_html_text, 'lxml')

#find street names in the map:
map_ads=soup_map.find_all('td', { "valign": "top" })


# ad = map_ads[1]
count_pages=[]
[count_pages.append(a) for a in soup_map.find_all('a', href=True) if a['href'].startswith(f'/{alphabet_letter}/')]
len(count_pages) 
count_pages


alphabet_letter='z'
list_starting_withZ=[]
for i in range(len(count_pages)):
    if i==0:
        streetmap_url=london_streetmap+alphabet_letter+'/'
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withZ.append(a.get_text(strip=True))
    else:
        streetmap_url=london_streetmap+alphabet_letter+'/'+str(i+1)
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })

        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withZ.append(a.get_text(strip=True))
list_starting_withZ=list(set(list_starting_withZ))
len(set(list_starting_withZ))
df_z=pd.DataFrame(list_starting_withZ, columns=['street_name_z']) 
df_z

[None, None, None, None, None, None]

6

[<a href="/a/" style="color:#000">[ 1 ]</a>,
 <a href="/a/2" style="color:#AAA">[ 2 ]</a>,
 <a href="/a/3" style="color:#AAA">[ 3 ]</a>,
 <a href="/a/4" style="color:#AAA">[ 4 ]</a>,
 <a href="/a/5" style="color:#AAA">[ 5 ]</a>,
 <a href="/a/6" style="color:#AAA">[ 6 ]</a>]

page 0   http://london.streetmapof.co.uk/z/
page 1   http://london.streetmapof.co.uk/z/2
page 2   http://london.streetmapof.co.uk/z/3
page 3   http://london.streetmapof.co.uk/z/4
page 4   http://london.streetmapof.co.uk/z/5
page 5   http://london.streetmapof.co.uk/z/6


8

,street_name_z
0,Zoffany Street
1,Zander Court
2,Zealand Road
3,Zoar Street
4,Zenoria Street
5,Zangwill Road
6,Zetland Street
7,Zennor Road


In [135]:
%%time
#get the results of EPC ratings for all the street names in London starting with Z:
#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in df_z['street_name_z'].tolist():
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)


for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
df_average_epc=pd.DataFrame(epc_avg, columns=['average_epc'])
df_average_epc
# avg_epc_tuple=list(zip(list_starting_withZ,epc_avg))
street_name_epc=pd.concat([df_z, df_average_epc],axis=1)
street_name_epc

CPU times: user 383 ms, sys: 35.9 ms, total: 419 ms
Wall time: 1min


,street_name_z,average_epc
0,Zoffany Street,B
1,Zander Court,C
2,Zealand Road,D
3,Zoar Street,0
4,Zenoria Street,D
5,Zangwill Road,D
6,Zetland Street,D
7,Zennor Road,0


In [137]:
%%time
#FINAL for the bigger for loop to collect all the elements of the list for letter "A" because it's a short list


london_streetmap='http://london.streetmapof.co.uk/'
alphabet_letter='a'
streetmap_url=london_streetmap+alphabet_letter+'/'

    
map_html_text = requests.get(streetmap_url).text
soup_map = BeautifulSoup(map_html_text, 'lxml')

#find street names in the map:
map_ads=soup_map.find_all('td', { "valign": "top" })


# ad = map_ads[1]
count_pages=[]
[count_pages.append(a) for a in soup_map.find_all('a', href=True) if a['href'].startswith(f'/{alphabet_letter}/')]
len(count_pages) 
count_pages


alphabet_letter='a'
list_starting_withA=[]
for i in range(len(count_pages)):
    if i==0:
        streetmap_url=london_streetmap+alphabet_letter+'/'
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withA.append(a.get_text(strip=True))
    else:
        streetmap_url=london_streetmap+alphabet_letter+'/'+str(i+1)
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })

        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withA.append(a.get_text(strip=True))
list_starting_withA=list(set(list_starting_withA))
len(set(list_starting_withA))
df_a=pd.DataFrame(list_starting_withA, columns=['street_name_a']) 
df_a


#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in df_a['street_name_a'].tolist():
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)


for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
df_average_epc_a=pd.DataFrame(epc_avg, columns=['average_epc'])
df_average_epc_a
# avg_epc_tuple=list(zip(list_starting_withZ,epc_avg))
street_name_epc_a=pd.concat([df_a, df_average_epc_a],axis=1)
street_name_epc_a

page 0   http://london.streetmapof.co.uk/a/
page 1   http://london.streetmapof.co.uk/a/2
page 2   http://london.streetmapof.co.uk/a/3
page 3   http://london.streetmapof.co.uk/a/4
page 4   http://london.streetmapof.co.uk/a/5
page 5   http://london.streetmapof.co.uk/a/6
CPU times: user 1min 19s, sys: 3.1 s, total: 1min 22s
Wall time: 2h 25min 59s


,street_name_a,average_epc
0,Avonmouth Street,D
1,Ainger Mews,E
2,Angell Road,C
3,Aldred Road,D
4,Alyth Gardens,D
...,...,...
1154,Ashburn Place,C
1155,Ashwood Road,D
1156,Amity Road,D
1157,Ashby Road,D


In [138]:
%%time
#FINAL for the bigger for loop to collect all the elements of the list for letter "B" because it's a short list


london_streetmap='http://london.streetmapof.co.uk/'
alphabet_letter='b'
streetmap_url=london_streetmap+alphabet_letter+'/'

    
map_html_text = requests.get(streetmap_url).text
soup_map = BeautifulSoup(map_html_text, 'lxml')

#find street names in the map:
map_ads=soup_map.find_all('td', { "valign": "top" })


# ad = map_ads[1]
count_pages=[]
[count_pages.append(a) for a in soup_map.find_all('a', href=True) if a['href'].startswith(f'/{alphabet_letter}/')]
len(count_pages) 
count_pages


alphabet_letter='b'
list_starting_withB=[]
for i in range(len(count_pages)):
    if i==0:
        streetmap_url=london_streetmap+alphabet_letter+'/'
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withB.append(a.get_text(strip=True))
    else:
        streetmap_url=london_streetmap+alphabet_letter+'/'+str(i+1)
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })

        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withB.append(a.get_text(strip=True))
list_starting_withB=list(set(list_starting_withB))
len(set(list_starting_withB))
df_b=pd.DataFrame(list_starting_withB, columns=['street_name_b']) 
df_b


#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in df_b['street_name_b'].tolist():
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)


for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
df_average_epc_b=pd.DataFrame(epc_avg, columns=['average_epc'])
df_average_epc_b
# avg_epc_tuple=list(zip(list_starting_withZ,epc_avg))
street_name_epc_b=pd.concat([df_b, df_average_epc_b],axis=1)
street_name_epc_b

page 0   http://london.streetmapof.co.uk/b/
page 1   http://london.streetmapof.co.uk/b/2
page 2   http://london.streetmapof.co.uk/b/3
page 3   http://london.streetmapof.co.uk/b/4
page 4   http://london.streetmapof.co.uk/b/5
page 5   http://london.streetmapof.co.uk/b/6
page 6   http://london.streetmapof.co.uk/b/7
page 7   http://london.streetmapof.co.uk/b/8
page 8   http://london.streetmapof.co.uk/b/9
page 9   http://london.streetmapof.co.uk/b/10
page 10   http://london.streetmapof.co.uk/b/11
page 11   http://london.streetmapof.co.uk/b/12
CPU times: user 2min 32s, sys: 5.51 s, total: 2min 38s
Wall time: 4h 28min 18s


,street_name_b,average_epc
0,Briscoe Road,D
1,Bigwood Road,D
2,Berkeley Walk,D
3,Bebbington Road,D
4,Bermondsey Square,B
...,...,...
2211,Ballast Quay,D
2212,Brunswick Crescent,D
2213,Beryl Road,D
2214,Broad Oak Close,C


In [139]:
%%time
#FINAL for the bigger for loop to collect all the elements of the list for letter "C" because it's a short list


london_streetmap='http://london.streetmapof.co.uk/'
alphabet_letter='c'
streetmap_url=london_streetmap+alphabet_letter+'/'

    
map_html_text = requests.get(streetmap_url).text
soup_map = BeautifulSoup(map_html_text, 'lxml')

#find street names in the map:
map_ads=soup_map.find_all('td', { "valign": "top" })


# ad = map_ads[1]
count_pages=[]
[count_pages.append(a) for a in soup_map.find_all('a', href=True) if a['href'].startswith(f'/{alphabet_letter}/')]
len(count_pages) 
count_pages


alphabet_letter='c'
list_starting_withC=[]
for i in range(len(count_pages)):
    if i==0:
        streetmap_url=london_streetmap+alphabet_letter+'/'
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })
        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withC.append(a.get_text(strip=True))
    else:
        streetmap_url=london_streetmap+alphabet_letter+'/'+str(i+1)
        print("page",i," ",streetmap_url)
        map_html_text = requests.get(streetmap_url).text
        soup_map = BeautifulSoup(map_html_text, 'lxml')
        map_ads=soup_map.find_all('td', { "valign": "top" })

        for k in range(len(map_ads)):
            ad = map_ads[k]


            for a in ad.find_all('a', href=True):
                if a['href'].startswith(f'/{alphabet_letter}/'):
                    count_pages.append(a)
                else:
                    list_starting_withC.append(a.get_text(strip=True))
list_starting_withC=list(set(list_starting_withC))
len(set(list_starting_withC))
df_c=pd.DataFrame(list_starting_withC, columns=['street_name_c']) 
df_c


#first create a list of URLs for scraping
list_epc_urls=[]
epc_url='https://find-energy-certificate.digital.communities.gov.uk/find-a-certificate/search-by-street-name-and-town?street_name='
town_url='&town=London'
epc_avg=[]

for street_name in df_c['street_name_c'].tolist():
    url_street_name=street_name.replace(' ','+')
    list_epc_urls.append(epc_url+url_street_name+town_url)


for url in list_epc_urls:
    
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    epc_ads=soup.find_all('tr', class_ = 'govuk-table__row')
    
    epc_id_ad = 0
    flat_address_final=[]
    epc_rating=[]
    for k in range(len(epc_ads)):
        ad = epc_ads[k]
        flat_address = ad.find_all('a', class_='govuk-link')
  
        if len(flat_address) !=0:
            flat_address_final.append(flat_address[0].get_text(strip=True))
        else: 
            continue

        epc_rating.append(ad.find_all('td', class_='govuk-table__cell')[0].get_text(strip=True))

        epc_id_ad += 1
    if len(epc_rating)!=0:
        epc_avg.append(mode(epc_rating))
    else:
        epc_avg.append(0)
df_average_epc_c=pd.DataFrame(epc_avg, columns=['average_epc'])
df_average_epc_c
# avg_epc_tuple=list(zip(list_starting_withZ,epc_avg))
street_name_epc_c=pd.concat([df_c, df_average_epc_c],axis=1)
street_name_epc_c

page 0   http://london.streetmapof.co.uk/c/
page 1   http://london.streetmapof.co.uk/c/2
page 2   http://london.streetmapof.co.uk/c/3
page 3   http://london.streetmapof.co.uk/c/4
page 4   http://london.streetmapof.co.uk/c/5
page 5   http://london.streetmapof.co.uk/c/6
page 6   http://london.streetmapof.co.uk/c/7
page 7   http://london.streetmapof.co.uk/c/8
page 8   http://london.streetmapof.co.uk/c/9
page 9   http://london.streetmapof.co.uk/c/10
page 10   http://london.streetmapof.co.uk/c/11
page 11   http://london.streetmapof.co.uk/c/12
page 12   http://london.streetmapof.co.uk/c/13
CPU times: user 2min 51s, sys: 5.54 s, total: 2min 56s
Wall time: 4h 57min 27s


,street_name_c,average_epc
0,Century Road,D
1,Crouch Hill,D
2,Caxton Street,B
3,Countess Road,D
4,Covington Gardens,D
...,...,...
2444,Colwith Road,D
2445,Cotherstone Road,D
2446,Cockerell Road,C
2447,Chewton Road,D


In [9]:
merged_table_epc_ratings=pd.read_csv('merged_table_epc_ratings.csv')
housing_scraper_dexter=pd.read_csv('housing_scraper_dexter.csv')

In [11]:
merged_table_epc_ratings
housing_scraper_dexter

,street_name,url,average_epc
0,Copse Hill,https://find-energy-certificate.digital.commun...,C
1,Fairhazel Gardens,https://find-energy-certificate.digital.commun...,D
2,Green Street,https://find-energy-certificate.digital.commun...,D
3,Kings Road,https://find-energy-certificate.digital.commun...,D
4,Ladbroke Road,https://find-energy-certificate.digital.commun...,D
5,Lancaster Gate,https://find-energy-certificate.digital.commun...,C
6,Old Queen Street,https://find-energy-certificate.digital.commun...,C
7,Philbeach Gardens,https://find-energy-certificate.digital.commun...,D
8,Queen Street,https://find-energy-certificate.digital.commun...,C
9,Queen Street,https://find-energy-certificate.digital.commun...,C


,ad_id,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,tube_line
0,151907,Lancaster Gate,23000000.0,"Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26,Central
1,124656,Wilton Crescent,19500000.0,"Wilton Crescent, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22,Piccadilly
2,138546,Whitehall Place,9250000.0,"Whitehall, SW1A",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12,Bakerloo
3,143179,Old Queen Street,9250000.0,"Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16,Circle
4,138545,Whitehall Place,9000000.0,"Whitehall, SW1A",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12,Bakerloo
5,138542,Whitehall Place,8500000.0,"Whitehall, SW1A",2,2,2,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12,Bakerloo
6,155928,Radnor Terrace,6500000.0,"Kensington, W14",4,4,1,https://www.dexters.co.uk/property-for-sale/fl...,Kensington (Olympia),0.22,District
7,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,Jubilee
8,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,DLR
9,159717,Ladbroke Road,5500000.0,"Notting Hill, W11",5,4,2,https://www.dexters.co.uk/property-for-sale/pr...,Notting Hill Gate,0.14,Central


In [12]:
dexters_epc=pd.merge(housing_scraper_dexter,merged_table_epc_ratings, on='street_name')

In [13]:
dexters_epc=dexters_epc.drop('url', axis=1)
dexters_epc

,ad_id,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,tube_line,url,average_epc
0,151907,Lancaster Gate,23000000.0,"Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26,Central,https://find-energy-certificate.digital.commun...,C
1,124656,Wilton Crescent,19500000.0,"Wilton Crescent, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22,Piccadilly,https://find-energy-certificate.digital.commun...,D
2,143179,Old Queen Street,9250000.0,"Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16,Circle,https://find-energy-certificate.digital.commun...,C
3,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,Jubilee,https://find-energy-certificate.digital.commun...,C
4,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,Jubilee,https://find-energy-certificate.digital.commun...,C
5,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,DLR,https://find-energy-certificate.digital.commun...,C
6,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,DLR,https://find-energy-certificate.digital.commun...,C
7,159717,Ladbroke Road,5500000.0,"Notting Hill, W11",5,4,2,https://www.dexters.co.uk/property-for-sale/pr...,Notting Hill Gate,0.14,Central,https://find-energy-certificate.digital.commun...,D
8,156629,Fairhazel Gardens,5495000.0,"South Hampstead, NW6",5,5,3,https://www.dexters.co.uk/property-for-sale/pr...,Finchley Road,0.19,Jubilee,https://find-energy-certificate.digital.commun...,D
9,157041,Kings Road,5000000.0,"Richmond, TW10",8,3,3,https://www.dexters.co.uk/property-for-sale/pr...,Richmond,0.41,District,https://find-energy-certificate.digital.commun...,D


In [14]:
dexters_epc=dexters_epc.drop('url', axis=1)
dexters_epc

,ad_id,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,tube_line,average_epc
0,151907,Lancaster Gate,23000000.0,"Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26,Central,C
1,124656,Wilton Crescent,19500000.0,"Wilton Crescent, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22,Piccadilly,D
2,143179,Old Queen Street,9250000.0,"Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16,Circle,C
3,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,Jubilee,C
4,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,Jubilee,C
5,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,DLR,C
6,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,DLR,C
7,159717,Ladbroke Road,5500000.0,"Notting Hill, W11",5,4,2,https://www.dexters.co.uk/property-for-sale/pr...,Notting Hill Gate,0.14,Central,D
8,156629,Fairhazel Gardens,5495000.0,"South Hampstead, NW6",5,5,3,https://www.dexters.co.uk/property-for-sale/pr...,Finchley Road,0.19,Jubilee,D
9,157041,Kings Road,5000000.0,"Richmond, TW10",8,3,3,https://www.dexters.co.uk/property-for-sale/pr...,Richmond,0.41,District,D


In [15]:
dexters_epc.to_csv('dexters_epc.csv')

In [29]:
#working on prepping data to be cleaned and encoded
dexters_epc_ratings=pd.read_csv('dexters_epc_ratings.csv')
dexters_epc_ratings

,ad_id,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,tube_line,average_epc
0,151907,Lancaster Gate,23000000.0,"Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26,Central,C
1,124656,Wilton Crescent,19500000.0,"Wilton Crescent, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22,Piccadilly,D
2,143179,Old Queen Street,9250000.0,"Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16,Circle,C
3,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,Jubilee,C
4,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,Jubilee,C
5,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,DLR,C
6,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,DLR,C
7,159717,Ladbroke Road,5500000.0,"Notting Hill, W11",5,4,2,https://www.dexters.co.uk/property-for-sale/pr...,Notting Hill Gate,0.14,Central,D
8,156629,Fairhazel Gardens,5495000.0,"South Hampstead, NW6",5,5,3,https://www.dexters.co.uk/property-for-sale/pr...,Finchley Road,0.19,Jubilee,D
9,157041,Kings Road,5000000.0,"Richmond, TW10",8,3,3,https://www.dexters.co.uk/property-for-sale/pr...,Richmond,0.41,District,D


In [21]:
#define efficient as A and B ratings 
efficient_ratings=['A','B']
dexters_epc_ratings['efficient'] = np.where(dexters_epc_ratings['average_epc'].isin(efficient_ratings), 1, 0)

In [24]:
dexters_epc_ratings.describe()

,ad_id,price,bedrooms,bathrooms,reception,distance,efficient
count,13.000000,1.300000e+01,13.000000,13.000000,13.000000,13.000000,13.0
mean,141766.153846,8.180385e+06,8.846154,3.307692,2.538462,0.283846,0.0
std,20120.541236,5.954267e+06,15.453404,1.250641,1.506397,0.297364,0.0
min,84720.000000,4.650000e+06,3.000000,0.000000,1.000000,0.100000,0.0
25%,138689.000000,5.000000e+06,3.000000,3.000000,2.000000,0.170000,0.0
50%,143179.000000,6.000000e+06,5.000000,3.000000,2.000000,0.180000,0.0
75%,156629.000000,6.000000e+06,5.000000,4.000000,3.000000,0.260000,0.0
max,159717.000000,2.300000e+07,60.000000,5.000000,6.000000,1.240000,0.0


In [25]:
dexters_epc_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ad_id           13 non-null     int64  
 1   street_name     13 non-null     object 
 2   price           13 non-null     float64
 3   address         13 non-null     object 
 4   bedrooms        13 non-null     int64  
 5   bathrooms       13 non-null     int64  
 6   reception       13 non-null     int64  
 7   link            13 non-null     object 
 8   subway_station  13 non-null     object 
 9   distance        13 non-null     float64
 10  tube_line       13 non-null     object 
 11  average_epc     13 non-null     object 
 12  efficient       13 non-null     int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1.4+ KB


In [26]:
dexters_epc_ratings = pd.get_dummies(data=dexters_epc_ratings, columns=['average_epc', "tube_line"])

In [27]:
dexters_epc_ratings

,ad_id,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,efficient,average_epc_C,average_epc_D,tube_line_Central,tube_line_Circle,tube_line_DLR,tube_line_District,tube_line_Jubilee,tube_line_Piccadilly
0,151907,Lancaster Gate,23000000.0,"Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26,0,1,0,1,0,0,0,0,0
1,124656,Wilton Crescent,19500000.0,"Wilton Crescent, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22,0,0,1,0,0,0,0,0,1
2,143179,Old Queen Street,9250000.0,"Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16,0,1,0,0,1,0,0,0,0
3,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,0,1,0,0,0,0,0,1,0
4,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,0,1,0,0,0,0,0,1,0
5,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,0,1,0,0,0,1,0,0,0
6,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,0,1,0,0,0,1,0,0,0
7,159717,Ladbroke Road,5500000.0,"Notting Hill, W11",5,4,2,https://www.dexters.co.uk/property-for-sale/pr...,Notting Hill Gate,0.14,0,0,1,1,0,0,0,0,0
8,156629,Fairhazel Gardens,5495000.0,"South Hampstead, NW6",5,5,3,https://www.dexters.co.uk/property-for-sale/pr...,Finchley Road,0.19,0,0,1,0,0,0,0,1,0
9,157041,Kings Road,5000000.0,"Richmond, TW10",8,3,3,https://www.dexters.co.uk/property-for-sale/pr...,Richmond,0.41,0,0,1,0,0,0,1,0,0


In [28]:
def clean_df_for_ML(dexters_epc_ratings):
    efficient_ratings=['A','B']
    dexters_epc_ratings['efficient'] = np.where(dexters_epc_ratings['average_epc'].isin(efficient_ratings), 1, 0)
    dexters_epc_ratings = pd.get_dummies(data=dexters_epc_ratings, columns=['average_epc', "tube_line"])
    return dexters_epc_ratings

In [30]:
clean_df_for_ML(dexters_epc_ratings)

,ad_id,street_name,price,address,bedrooms,bathrooms,reception,link,subway_station,distance,efficient,average_epc_C,average_epc_D,tube_line_Central,tube_line_Circle,tube_line_DLR,tube_line_District,tube_line_Jubilee,tube_line_Piccadilly
0,151907,Lancaster Gate,23000000.0,"Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26,0,1,0,1,0,0,0,0,0
1,124656,Wilton Crescent,19500000.0,"Wilton Crescent, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22,0,0,1,0,0,0,0,0,1
2,143179,Old Queen Street,9250000.0,"Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16,0,1,0,0,1,0,0,0,0
3,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,0,1,0,0,0,0,0,1,0
4,138689,Queen Street,6000000.0,"Mayfair, W1J",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Green Park,0.18,0,1,0,0,0,0,0,1,0
5,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,0,1,0,0,0,1,0,0,0
6,138690,Queen Street,6000000.0,"Mayfair, W1J",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,East India,0.17,0,1,0,0,0,1,0,0,0
7,159717,Ladbroke Road,5500000.0,"Notting Hill, W11",5,4,2,https://www.dexters.co.uk/property-for-sale/pr...,Notting Hill Gate,0.14,0,0,1,1,0,0,0,0,0
8,156629,Fairhazel Gardens,5495000.0,"South Hampstead, NW6",5,5,3,https://www.dexters.co.uk/property-for-sale/pr...,Finchley Road,0.19,0,0,1,0,0,0,0,1,0
9,157041,Kings Road,5000000.0,"Richmond, TW10",8,3,3,https://www.dexters.co.uk/property-for-sale/pr...,Richmond,0.41,0,0,1,0,0,0,1,0,0
